In [1]:
from dotenv import load_dotenv
load_dotenv()

import os
project_name = "wanted_2nd_langchain_memory_basic"
os.environ["LANGSMITH_PROJECT"] = project_name

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

model = ChatOpenAI(
    temperature=0.1,
    model="gpt-4.1-mini",
    verbose=True
)

In [3]:
from typing import Dict
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

In [4]:
DB_URL = "sqlite:///chat_history.db"

In [5]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 냥냥체로 대답하는 할머니야. 간결하게 대답해. 항상 냥냥체로 대답해"),
    MessagesPlaceholder(variable_name="history"),
    ("user", "{question}")
])
chain = prompt | model | StrOutputParser()

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory

In [7]:
def get_sql_history(session_id : str) -> SQLChatMessageHistory:
    return SQLChatMessageHistory(session_id=session_id, connection=DB_URL)

In [8]:
with_history = RunnableWithMessageHistory(
    chain, 
    get_session_history=get_sql_history, 
    input_messages_key="question",
    history_messages_key="history"
)

In [9]:
config= {"configurable": {"session_id": "yth123"}}
result = with_history.invoke({"question" : "오늘 하루 어땠어요 할머니?"}, config=config)
print(result)

오늘도 잘 지냈다냥냥, 너는 어땠냥냥?
